In [ ]:
import os
import random
import gym
import time
import pylab
import numpy as np
from collections import deque
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense, Lambda, Add, Conv2D, Flatten, Reshape, LSTM
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras import backend as K
import tensorflow_probability as tfp
import cv2
from threading import Thread, Lock
import threading
from threading import Thread, Lock

import vtrace

tfd = tfp.distributions

os.environ['CUDA_VISIBLE_DEVICES'] = '-1'


class OurModel(tf.keras.Model):
    def __init__(self, input_shape, action_space):
        super(OurModel, self).__init__()
        
        self.dense_0 = Dense(128, activation='relu', kernel_initializer='lecun_normal')
        self.dense_1 = Dense(action_space, kernel_initializer='lecun_normal')
        self.dense_2 = Dense(1, kernel_initializer='lecun_normal')
        
    def call(self, X_input):
        X_input = self.dense_0(X_input)
        action_prob = self.dense_1(X_input)
        value = self.dense_2(X_input)
        
        return action_prob, value


def safe_log(x):
  return tf.where(
      tf.math.equal(x, 0),
      tf.zeros_like(x),
      tf.math.log(tf.math.maximum(1e-12, x)))


def take_vector_elements(vectors, indices):
    return tf.gather_nd(vectors, tf.stack([tf.range(tf.shape(vectors)[0]), indices], axis=1))
    

class IMPALA_Agent:
    def __init__(self, env_name):    
        self.env = gym.make(env_name)
        self.env_name = env_name       
        self.action_size = self.env.action_space.n
        self.EPISODES, self.episode, self.max_average = 200000, 0, 190.0 # specific for cartpole
        
        # Instantiate memory
        memory_size = 5000
        self.memory = []
        self.lock = Lock()

        self.batch_size = 32

        self.Save_Path = 'Models'
        if not os.path.exists(self.Save_Path): os.makedirs(self.Save_Path)
        self.scores, self.episodes, self.average = [], [], []

        self.Model_name = os.path.join(self.Save_Path, self.env_name + "_IMPALA.h5")
        self.path = '{}_IMPALA'.format(self.env_name)
        
        self.COLS = 4
        
        self.state_size = (self.COLS)
        self.image_memory = np.zeros(self.state_size)
        
        self.model = self.make_model('Model', self.state_size, self.action_size)
        
        #self.model.summary()
        self.learning_rate = 0.001
        self.optimizer = tf.keras.optimizers.Adam(self.learning_rate)
        
    def remember(self, state, action, policy, reward, next_state, done):
        experience = state, action, policy, reward, next_state, done
        self.memory.append((experience))

    def act(self, state):
        prediction = self.model(state, training=False)
        
        dist = tfd.Categorical(logits=prediction[0])
        action = int(dist.sample()[0])
        policy = prediction[0]
        
        return action, policy
    
    def make_model(self, name, input_shape, action_space):
        state = tf.keras.Input(shape=input_shape)
        head = OurModel(input_shape, action_space)(state)
        model = tf.keras.Model(inputs=state, outputs=head, name=name)
        
        return model
    
    #@tf.function
    def update(self, states, actions, agent_policies, rewards, next_states, dones):
        online_variables = self.model.trainable_variables
        with tf.GradientTape() as tape:
            tape.watch(online_variables)
            
            learner_outputs = self.model(states, training=True)
            
            agent_logits = tf.nn.softmax(agent_policies[:-1])
            actions = actions[:-1]
            rewards = rewards[1:]
            dones = dones[1:]
        
            learner_policies = learner_outputs[0]
            learner_logits = tf.nn.softmax(learner_policies[:-1])
            
            learner_values = learner_outputs[1]
            learner_values = tf.squeeze(learner_values)
            
            bootstrap_value = learner_values[-1]
            learner_values = learner_values[:-1]
            
            discounting = 0.99
            discounts = tf.cast(~dones, tf.float32) * discounting
            
            actions = tf.convert_to_tensor(actions, dtype=tf.int32)
                
            target_action_probs = take_vector_elements(learner_logits, actions)
            target_action_log_probs = tf.math.log(target_action_probs)
            
            behaviour_action_probs = take_vector_elements(agent_logits, actions)
            behaviour_action_log_probs = tf.math.log(behaviour_action_probs)
            
            lambda_ = 1.0
            
            log_rhos = target_action_log_probs - behaviour_action_log_probs
            
            log_rhos = tf.convert_to_tensor(log_rhos, dtype=tf.float32)
            discounts = tf.convert_to_tensor(discounts, dtype=tf.float32)
            rewards = tf.convert_to_tensor(rewards, dtype=tf.float32)
            values = tf.convert_to_tensor(learner_values, dtype=tf.float32)
            bootstrap_value = tf.convert_to_tensor(bootstrap_value, dtype=tf.float32)
            
            clip_rho_threshold = tf.convert_to_tensor(1.0, dtype=tf.float32)
            clip_pg_rho_threshold = tf.convert_to_tensor(1.0, dtype=tf.float32)
            
            rhos = tf.math.exp(log_rhos)
            
            clipped_rhos = tf.minimum(clip_rho_threshold, rhos, name='clipped_rhos')
            
            cs = tf.minimum(1.0, rhos, name='cs')
            cs *= tf.convert_to_tensor(lambda_, dtype=tf.float32)

            values_t_plus_1 = tf.concat([values[1:], tf.expand_dims(bootstrap_value, 0)], axis=0)
            deltas = clipped_rhos * (rewards + discounts * values_t_plus_1 - values)
        
            acc = tf.zeros_like(bootstrap_value)
            vs_minus_v_xs = []
            for i in range(int(discounts.shape[0]) - 1, -1, -1):
                discount, c, delta = discounts[i], cs[i], deltas[i]
                acc = delta + discount * c * acc
                vs_minus_v_xs.append(acc)  
            
            vs_minus_v_xs = vs_minus_v_xs[::-1]
            
            vs = tf.add(vs_minus_v_xs, values, name='vs')
            vs_t_plus_1 = tf.concat([vs[1:], tf.expand_dims(bootstrap_value, 0)], axis=0)
            clipped_pg_rhos = tf.minimum(clip_pg_rho_threshold, rhos, name='clipped_pg_rhos')
            
            pg_advantages = (clipped_pg_rhos * (rewards + discounts * vs_t_plus_1 - values))
            
            vs = tf.stop_gradient(vs)
            pg_advantages = tf.stop_gradient(pg_advantages)
            
            actor_loss = -tf.reduce_mean(target_action_log_probs * pg_advantages)
            
            baseline_cost = 0.5
            v_error = values - vs
            critic_loss = baseline_cost * 0.5 * tf.reduce_mean(tf.square(v_error))
            
            total_loss = actor_loss + critic_loss

        grads = tape.gradient(total_loss, self.model.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.model.trainable_variables))
    
    def replay(self):
        memory_len = len(self.memory)
        if len(self.memory) > self.batch_size:
            start_index = random.randint(0, memory_len - self.batch_size)
            minibatch = self.memory[start_index:start_index+self.batch_size]
        else:
            return

        state = np.zeros((self.batch_size, self.state_size), dtype=np.float32)
        action = np.zeros(self.batch_size, dtype=np.int32)
        policy = np.zeros((self.batch_size, self.action_size), dtype=np.float32)
        reward = np.zeros(self.batch_size, dtype=np.float32)
        next_state = np.zeros((self.batch_size, self.state_size), dtype=np.float32)
        done = np.zeros(self.batch_size, dtype=np.bool)
      
        for i in range(len(minibatch)):
            state[i], action[i], policy[i], reward[i], next_state[i], done[i] = minibatch[i]

        self.update(state, action, policy, reward, next_state, done)
        
    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)

    pylab.figure(figsize=(18, 9))
    def PlotModel(self, score, episode):
        self.scores.append(score)
        self.episodes.append(episode)
        self.average.append(sum(self.scores[-50:]) / len(self.scores[-50:]))
        pylab.plot(self.episodes, self.average, 'r')
        pylab.plot(self.episodes, self.scores, 'b')
        pylab.ylabel('Score', fontsize=18)
        pylab.xlabel('Games', fontsize=18)
        try:
            pylab.savefig(self.env_name + "_IMAPLA.png")
        except OSError:
            pass

        # no need to worry about model, when doing a lot of experiments
        self.Model_name = os.path.join(self.Save_Path, self.env_name + "_IMAPLA.h5")

        return self.average[-1]

    def imshow(self, image, rem_step=0):
        cv2.imshow("cartpole" + str(rem_step), image[:,:,rem_step])
        if cv2.waitKey(25) & 0xFF == ord("q"):
            cv2.destroyAllWindows()
        
    def reset(self, env):
        state = env.reset()
            
        return state

    def step(self, action, env):
        next_state, reward, done, info = env.step(action)
        
        return next_state, reward, done, info
    
    def run(self, n_threads):
        self.env.close()
        
        # Instantiate one environment per thread
        envs = [gym.make(self.env_name) for i in range(n_threads)]

        # Create threads
        threads = [threading.Thread(
                target=self.run_threading,
                daemon=True,
                args=(self,
                    envs[i],
                    i)) for i in range(n_threads)]

        for t in threads:
            time.sleep(2)
            t.start()
            
        for t in threads:
            time.sleep(10)
            t.join()
    
    def run_threading(self, agent, env, thread):
        max_average = 190.0
        total_step = 0
        for e in range(self.EPISODES):
            state = self.reset(env)
            state = np.expand_dims(state, 0)
            
            done = False
            score = 0
            SAVING = ''
            while not done:
                #self.env.render()
                action, policy = self.act(state)
                
                next_state, reward, done, _ = self.step(action, env)
                next_state = np.expand_dims(next_state, 0)
                
                self.remember(state, action, policy, reward / 200.0, next_state, done)
                state = next_state
                score += reward

                if done:           
                    break
                
                #self.lock.acquire()
                if thread == 0:
                    #if total_step % 8 == 0:
                        # train model
                    self.replay()
                    #self.lock.release()
                    
                    total_step += 1
                
            # Update episode count
            with self.lock:
                average = self.PlotModel(score, self.episode)
                # saving best models
                if average >= self.max_average:
                    self.max_average = average
                    #self.save()
                    SAVING = "SAVING"
                else:
                    SAVING = ""

                print("episode: {}/{}, thread: {}, score: {}, average: {:.2f} {}".format(self.episode, self.EPISODES, thread, score, average, SAVING))
                if(self.episode < self.EPISODES):
                    self.episode += 1
                    
        # close environemnt when finish training
        self.env.close()

    def test(self, Model_name):
        self.load(Model_name)
        for e in range(self.EPISODES):
            state = self.reset()
            done = False
            score = 0
            while not done:
                self.env.render()
                action = np.argmax(self.model.predict(state))
                state, reward, done, _ = self.step(action)
                score += reward
                if done:
                    print("episode: {}/{}, score: {}".format(e, self.EPISODES, score))
                    break
                    
        self.env.close()

        
if __name__ == "__main__":
    env_name = 'CartPole-v0'
    agent = IMPALA_Agent(env_name)
    agent.run(n_threads=8)
    #agent.test('Models/Pong-v0_DDQN_CNN.h5')

episode: 0/200000, thread: 0, score: 18.0, average: 18.00 
episode: 1/200000, thread: 0, score: 31.0, average: 24.50 
episode: 2/200000, thread: 0, score: 32.0, average: 27.00 
episode: 3/200000, thread: 0, score: 9.0, average: 22.50 
episode: 4/200000, thread: 1, score: 57.0, average: 29.40 
episode: 5/200000, thread: 1, score: 31.0, average: 29.67 
episode: 6/200000, thread: 1, score: 26.0, average: 29.14 
episode: 7/200000, thread: 1, score: 18.0, average: 27.75 
episode: 8/200000, thread: 1, score: 40.0, average: 29.11 
episode: 9/200000, thread: 1, score: 21.0, average: 28.30 
episode: 10/200000, thread: 1, score: 25.0, average: 28.00 
episode: 11/200000, thread: 1, score: 89.0, average: 33.08 
episode: 12/200000, thread: 1, score: 23.0, average: 32.31 
episode: 13/200000, thread: 0, score: 73.0, average: 35.21 
episode: 14/200000, thread: 1, score: 40.0, average: 35.53 
episode: 15/200000, thread: 2, score: 22.0, average: 34.69 
episode: 16/200000, thread: 1, score: 22.0, average

episode: 136/200000, thread: 6, score: 113.0, average: 57.66 
episode: 137/200000, thread: 4, score: 30.0, average: 57.42 
episode: 138/200000, thread: 1, score: 176.0, average: 59.92 
episode: 139/200000, thread: 7, score: 91.0, average: 60.68 
episode: 140/200000, thread: 5, score: 52.0, average: 61.22 
episode: 141/200000, thread: 3, score: 43.0, average: 60.52 
episode: 142/200000, thread: 6, score: 18.0, average: 59.82 
episode: 143/200000, thread: 2, score: 46.0, average: 58.02 
episode: 144/200000, thread: 4, score: 20.0, average: 57.68 
episode: 145/200000, thread: 1, score: 36.0, average: 57.76 
episode: 146/200000, thread: 5, score: 44.0, average: 57.98 
episode: 147/200000, thread: 7, score: 86.0, average: 58.64 
episode: 148/200000, thread: 3, score: 37.0, average: 58.22 
episode: 149/200000, thread: 6, score: 20.0, average: 57.66 
episode: 150/200000, thread: 4, score: 49.0, average: 57.16 
episode: 151/200000, thread: 1, score: 46.0, average: 56.02 
episode: 152/200000, t

episode: 270/200000, thread: 2, score: 87.0, average: 103.82 
episode: 271/200000, thread: 7, score: 70.0, average: 102.26 
episode: 272/200000, thread: 3, score: 87.0, average: 102.88 
episode: 273/200000, thread: 5, score: 116.0, average: 104.68 
episode: 274/200000, thread: 6, score: 102.0, average: 106.00 
episode: 275/200000, thread: 1, score: 132.0, average: 105.64 
episode: 276/200000, thread: 7, score: 60.0, average: 104.24 
episode: 277/200000, thread: 4, score: 192.0, average: 106.48 
episode: 278/200000, thread: 2, score: 170.0, average: 109.26 
episode: 279/200000, thread: 3, score: 131.0, average: 110.28 
episode: 280/200000, thread: 6, score: 59.0, average: 107.46 
episode: 281/200000, thread: 5, score: 103.0, average: 105.52 
episode: 282/200000, thread: 1, score: 27.0, average: 104.76 
episode: 283/200000, thread: 7, score: 98.0, average: 105.96 
episode: 284/200000, thread: 2, score: 81.0, average: 105.86 
episode: 285/200000, thread: 4, score: 133.0, average: 107.22 


episode: 402/200000, thread: 5, score: 200.0, average: 136.46 
episode: 403/200000, thread: 1, score: 198.0, average: 136.42 
episode: 404/200000, thread: 2, score: 134.0, average: 137.24 
episode: 405/200000, thread: 6, score: 181.0, average: 139.40 
episode: 406/200000, thread: 7, score: 166.0, average: 140.00 
episode: 407/200000, thread: 3, score: 113.0, average: 140.56 
episode: 408/200000, thread: 4, score: 200.0, average: 142.68 
episode: 409/200000, thread: 5, score: 200.0, average: 145.16 
episode: 410/200000, thread: 1, score: 158.0, average: 146.14 
episode: 411/200000, thread: 2, score: 200.0, average: 148.70 
episode: 412/200000, thread: 6, score: 200.0, average: 150.02 
episode: 413/200000, thread: 3, score: 119.0, average: 148.40 
episode: 414/200000, thread: 7, score: 200.0, average: 150.88 
episode: 415/200000, thread: 5, score: 67.0, average: 148.22 
episode: 416/200000, thread: 4, score: 200.0, average: 148.72 
episode: 417/200000, thread: 1, score: 167.0, average: 1

episode: 533/200000, thread: 3, score: 116.0, average: 170.46 
episode: 534/200000, thread: 2, score: 124.0, average: 171.40 
episode: 535/200000, thread: 6, score: 129.0, average: 169.98 
episode: 536/200000, thread: 5, score: 200.0, average: 169.98 
episode: 537/200000, thread: 4, score: 200.0, average: 171.20 
episode: 538/200000, thread: 1, score: 116.0, average: 169.52 
episode: 539/200000, thread: 0, score: 67.0, average: 166.86 
episode: 540/200000, thread: 7, score: 200.0, average: 166.86 
episode: 541/200000, thread: 3, score: 200.0, average: 169.22 
episode: 542/200000, thread: 2, score: 162.0, average: 168.46 
episode: 543/200000, thread: 6, score: 200.0, average: 168.46 
episode: 544/200000, thread: 5, score: 109.0, average: 166.64 
episode: 545/200000, thread: 4, score: 200.0, average: 166.64 
episode: 546/200000, thread: 1, score: 183.0, average: 166.30 
episode: 547/200000, thread: 7, score: 137.0, average: 165.04 
episode: 548/200000, thread: 3, score: 179.0, average: 1

episode: 664/200000, thread: 7, score: 200.0, average: 185.56 
episode: 665/200000, thread: 6, score: 200.0, average: 188.10 
episode: 666/200000, thread: 2, score: 200.0, average: 188.10 
episode: 667/200000, thread: 4, score: 200.0, average: 188.10 
episode: 668/200000, thread: 5, score: 200.0, average: 190.82 SAVING
episode: 669/200000, thread: 1, score: 200.0, average: 190.82 SAVING
episode: 670/200000, thread: 3, score: 200.0, average: 190.82 SAVING
episode: 671/200000, thread: 7, score: 200.0, average: 190.82 SAVING
episode: 672/200000, thread: 6, score: 200.0, average: 190.92 SAVING
episode: 673/200000, thread: 2, score: 200.0, average: 192.92 SAVING
episode: 674/200000, thread: 4, score: 200.0, average: 195.38 SAVING
